In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.8 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

In [3]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
cyber_data = load_dataset('bhattbhavesh91/cyber-dataset')

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/48.8M [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/93664 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/31218 [00:00<?, ? examples/s]

In [6]:
cyber_data

DatasetDict({
    train: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels'],
        num_rows: 93664
    })
    test: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels'],
        num_rows: 31218
    })
})

In [7]:
def tokenize(batch):
    return tokenizer(
        batch["crimeaditionalinfo"],
        padding=True,
        truncation=True,
        max_length=512
    )

In [8]:
cyber_data_encoded = cyber_data.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/93664 [00:00<?, ? examples/s]

Map:   0%|          | 0/31218 [00:00<?, ? examples/s]

In [9]:
cyber_data_encoded

DatasetDict({
    train: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 93664
    })
    test: Dataset({
        features: ['category', 'sub_category', 'crimeaditionalinfo', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 31218
    })
})

In [10]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

cyber_data_encoded.set_format('tf',
                            columns=['input_ids',
                                     'attention_mask',
                                     'token_type_ids',
                                     'labels'])

BATCH_SIZE = 32

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(cyber_data_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(cyber_data_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [11]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[ 101, 6926, 4751, ...,    0,    0,    0],
       [ 101, 2288, 1037, ...,    0,    0,    0],
       [ 101, 2909, 2651, ...,    0,    0,    0],
       ...,
       [ 101, 2027, 2081, ...,    0,    0,    0],
       [ 101, 1045, 2001, ...,    0,    0,    0],
       [ 101, 2039, 2072, ...,    0,    0,    0]])>, 'attention_mask': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(32, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([1 1 0 1 3 0 7 0 3 1 4 3 1 1 3 1 1 1 4 1 1 1 3 1 1 1 1 1 1 8 

In [12]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [13]:
classifier = BERTForClassification(model, num_classes=14)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [14]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
2927/2927 [==============================] - 1265s 419ms/step - loss: 0.7697 - accuracy: 0.7455
Epoch 2/3
2927/2927 [==============================] - 1225s 419ms/step - loss: 0.6475 - accuracy: 0.7739
Epoch 3/3
2927/2927 [==============================] - 1226s 419ms/step - loss: 0.5981 - accuracy: 0.7914


In [15]:
classifier.evaluate(test_dataset)

976/976 [==============================] - 146s 146ms/step - loss: 0.6567 - accuracy: 0.7698


[0.6567122936248779, 0.769812285900116]

In [16]:
from huggingface_hub import notebook_login

notebook_login()

In [18]:
model.push_to_hub("bhattbhavesh91/cyber-query-classifier-v2")

tf_model.h5:   0%|          | 0.00/438M [00:00<?, ?B/s]